In [0]:
%python
storage_key = dbutils.secrets.get(scope="movie-scope", key="storage-access-key")
spark.conf.set(
    "fs.azure.account.key.errisbinxhija01.dfs.core.windows.net",
    storage_key
)

In [0]:
from pyspark.sql import functions as F

# 1. Në vend që të përdorim variabla, i lexojmë direkt nga tabelat Silver që krijuam
movies_s = spark.read.table("movies_db.movies_silver")
recs_s = spark.read.table("movies_db.recommendations_silver")
belief_s = spark.read.table("movies_db.belief_silver")

# 2. Bashkimi (Join)
# Bashkojmë Rekomandimet me Filmat
gold_raw = recs_s.join(movies_s, on="movieId", how="inner")

# Bashkojmë me Belief Data
gold_combined = gold_raw.join(
    belief_s, 
    on=["userId", "movieId"], 
    how="left"
)

# 3. Pastrimi i kolonave që nuk duhen për ML
# Heqim çdo gjë që nuk është numër ose feature e dobishme
cols_to_drop = [
    "title", "genres", "genres_list", "tstamp", "tstamp_raw", 
    "rec_timestamp", "processed_at", "watchDate"
]

gold_ml_ready = gold_combined.drop(*cols_to_drop).fillna(0)

# 4. Ruajtja finale në Gold
gold_ml_ready.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("movies_db.dataset_gold_xgboost")

print("✨ Shtresa GOLD u krijua me sukses duke përdorur tabelat e ruajtura!")
display(gold_ml_ready.limit(5))